### GPU, 라이브러리 활성화

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
from PIL import Image
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import load_model



In [ ]:
print(tf.__version__)
tf.test.is_gpu_available()

### 데이터파일 경로 설정

In [ ]:
train_dir = os.path.join('C:/data/')
labels = ['can', 'plastic']

#파일 경로에있는 이미지 수 확인 
for label in labels:
    directory = os.path.join(train_dir, label)
    print('Images of labels \"' + label + "\":", len(os.listdir(directory)))

### 그림 10개씩 확인해보기

In [ ]:
plt.figure(figsize=(30,14))

for i in range(2):
    directory = os.path.join(train_dir,labels[i]) #directory 경로 설정 
    for j in range(10):
        path = os.path.join(directory, os.listdir(directory)[j]) # path
        img = mpimg.imread(path)
        
        plt.subplot(2,10, i*10 + j +1)
        plt.imshow(img)
        
        if j ==0:
            plt.ylabel(labels[i], fontsize=20)
        
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]); 
plt.tight_layout()
plt.show()

# 그림 크기 설정 
# directory 설정(훈련데이터 경로까지)
# 

### 이미지 사이즈 반틈으로 줄이기
- 이름_half 로 저장됨
- 실행 X

In [ ]:
train_dir = 'C:/data/'
j = ['can_j', 'pla']
for i in range(2):
    files = glob.glob(train_dir + j[i] +"/*.jpg") # 경로 
    for f in files:
        img = Image.open(f)
        img_resize = img.resize((int(img.width/2), int(img.height/2)))
        title, ext = os.path.splitext(f)
        name = os.path.basename(f)
        img_resize.save('C:/data/can/'+name+"_half"+ext)
        print(labels[i] + str(i) + '저장완료')

#### 바뀐 이미지 크기 확인

In [ ]:
img = mpimg.imread(directory+"/can1_half.jpg")
img.shape

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(2, activation='softmax'),
])
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])


### 데이터증강

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    'C:/data/train',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

validation_generator = test_datagen.flow_from_directory(
    'C:/data/train',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

### Call back 함수

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.90):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

### Training

In [ ]:
history = model.fit_generator(train_generator, epochs =15, verbose=1, validation_data = validation_generator)

### Test

In [ ]:
# 테스트할 이미지 경로
image_path = '/content/gdrive/MyDrive/AIStudy/plastic.jpg'

# 이미지 크기 조정
target_size = (128, 128)
img = image.load_img(image_path, target_size=target_size)
img2 = image.load_img(image_path, target_size=target_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # 이미지를 0과 1 사이의 값으로 정규화

# 예측 수행
predictions = model.predict(img)

# 예측 결과 해석
predicted_class = np.argmax(predictions[0])

class_labels = {0: 'can', 1: 'plastic'}  # 클래스 레이블과 인덱스 매핑

# 예측 결과 해석
predicted_class_index = np.argmax(predictions[0])
predicted_class_label = class_labels[predicted_class_index]

# 결과 표시
print("예측된 클래스:", predicted_class_label)

plt.imshow(img2)
plt.xticks([])
plt.yticks([])
plt.title('Actual: {}      Pred: {}'.format(labels[1], predicted_class_label))

### 모델 불러오기

In [ ]:
model = tf.keras.models.load_model('C:/Users/jihun/Classification_using_cnn/my_model.h5', compile=False)

In [ ]:
# 테스트할 이미지 경로
image_path = '/content/gdrive/MyDrive/AIStudy/can.jpg'

# 이미지 크기 조정
target_size = (128, 128)
img = image.load_img(image_path, target_size=target_size)
img2 = image.load_img(image_path, target_size=target_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # 이미지를 0과 1 사이의 값으로 정규화

# 예측 수행
predictions = model.predict(img)

# 예측 결과 해석
predicted_class = np.argmax(predictions[0])

class_labels = {0: 'can', 1: 'plastic'}  # 클래스 레이블과 인덱스 매핑

# 예측 결과 해석
predicted_class_index = np.argmax(predictions[0])
predicted_class_label = class_labels[predicted_class_index]

# 결과 표시
print("예측된 클래스:", predicted_class_label)

plt.imshow(img2)
plt.xticks([])
plt.yticks([])
plt.title('Actual: {}      Pred: {}'.format(labels[0], predicted_class_label))

### 성능확인

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'b*-', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_loss, 'b*-', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_x, test_y = validation_generator.__getitem__(1)
# test_x is used to train the data
# test_y refers to the actual answer of the testing set data
# preds refers to the predicted class
preds = model.predict(test_x)

plt.figure(figsize=(20, 20))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])